In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fuzzywuzzy import process, fuzz

pd.set_option('display.max_columns', None)

In [56]:
table_complete = pd.read_csv("saiku-export.csv")

desempleo = pd.read_csv("Desempleo.csv")

pd.set_option('display.max_columns', None)#mostramos todas las columnas.

In [57]:
table_complete.head()

,País de nacimiento Agresor,Suicidio,Convivencia,Denuncia agresor,Tramo de edad Agresor,Tramo de edad Víctima,Comunidad autónoma,Pareja - Expareja,Año,Mes,Número de mujeres víctimas mortales,Número de hijos e hijas menores huérfanos
0,España,No hubo tentativa,Sí,No había denuncia,18-20 años,21-30 años,Andalucía,Pareja,Año 2012,Abril,1.0,0.0
1,España,No hubo tentativa,Sí,No había denuncia,18-20 años,21-30 años,Andalucía,Pareja en fase de separación,Año 2006,Diciembre,1.0,0.0
2,España,No hubo tentativa,Sí,No había denuncia,18-20 años,21-30 años,Cataluña,Pareja,Año 2009,Agosto,1.0,0.0
3,España,No hubo tentativa,Sí,No había denuncia,21-30 años,16-17 años,Andalucía,Pareja,Año 2023,Febrero,1.0,0.0
4,España,No hubo tentativa,Sí,No había denuncia,21-30 años,18-20 años,Comunidad de Madrid,Pareja,Año 2012,Noviembre,1.0,0.0


In [58]:
#Cambio de nombre columnas

names={"Comunidad autónoma":"comunidad_autonoma", "Tramo de edad Agresor":"edad_agresor","Tramo de edad Víctima":"edad_victima","Convivencia":"convivencia","Pareja - Expareja":"estado","Número de mujeres víctimas mortales":"n_victimas_mortales","Número de hijos e hijas menores huérfanos":"hijos_huerfanos", "País de nacimiento Agresor":"pais_nac_agresor","Suicidio":"suicidio","Año":"año", "Mes":"mes"}

In [59]:
table_complete.rename(columns=names, inplace=True) #aplicamos el cambio de nombres.

In [60]:
table_complete.shape

(1240, 12)

In [61]:
table_complete = table_complete.drop("Denuncia agresor", axis=1) #eliminamos tablas que no necesitamos.

In [62]:
table_complete.head()

,pais_nac_agresor,suicidio,convivencia,edad_agresor,edad_victima,comunidad_autonoma,estado,año,mes,n_victimas_mortales,hijos_huerfanos
0,España,No hubo tentativa,Sí,18-20 años,21-30 años,Andalucía,Pareja,Año 2012,Abril,1.0,0.0
1,España,No hubo tentativa,Sí,18-20 años,21-30 años,Andalucía,Pareja en fase de separación,Año 2006,Diciembre,1.0,0.0
2,España,No hubo tentativa,Sí,18-20 años,21-30 años,Cataluña,Pareja,Año 2009,Agosto,1.0,0.0
3,España,No hubo tentativa,Sí,21-30 años,16-17 años,Andalucía,Pareja,Año 2023,Febrero,1.0,0.0
4,España,No hubo tentativa,Sí,21-30 años,18-20 años,Comunidad de Madrid,Pareja,Año 2012,Noviembre,1.0,0.0


In [63]:
table_complete.info() #miramos tipos de datos de la tabla importada.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pais_nac_agresor     1240 non-null   object 
 1   suicidio             1240 non-null   object 
 2   convivencia          1240 non-null   object 
 3   edad_agresor         1240 non-null   object 
 4   edad_victima         1240 non-null   object 
 5   comunidad_autonoma   1240 non-null   object 
 6   estado               1240 non-null   object 
 7   año                  1240 non-null   object 
 8   mes                  1240 non-null   object 
 9   n_victimas_mortales  1240 non-null   float64
 10  hijos_huerfanos      1240 non-null   float64
dtypes: float64(2), object(9)
memory usage: 106.7+ KB


In [64]:
table_complete.año = table_complete.año.str.replace("Año", "") #reemplazamos Año por nada en las filas. 

In [66]:
table_complete.rename(columns={"comunidad_autonoma":"CCAA"}, inplace=True)

In [69]:
def merge_with_fuzzy_matching(df1, df2, column_name):
    # Obtener nombres únicos de 'CCAA' de ambos DataFrames
    nombres_df1 = df1[column_name].unique()
    nombres_df2 = df2[column_name].unique()
    
    # Mapear nombres de df1 a los nombres más similares de df2
    mapeo = {nombre_df1: process.extractOne(nombre_df1, nombres_df2)[0] for nombre_df1 in nombres_df1}
    
    # Reemplazar nombres de df1 con los de df2
    df1[column_name] = df1[column_name].map(mapeo)
    
    # Fusionar DataFrames utilizando la columna 'CCAA'
    df_merged = pd.merge(df1, df2, on=column_name)
    
    return df_merged

In [70]:
df_merged = merge_with_fuzzy_matching(table_complete, desempleo, 'CCAA')

In [75]:
df_merged.head()

,pais_nac_agresor,suicidio,convivencia,edad_agresor,edad_victima,CCAA,estado,año,mes,n_victimas_mortales,hijos_huerfanos,tasa de desempleo %
0,España,No hubo tentativa,Sí,18-20 años,21-30 años,Andalucía,Pareja,2012,Abril,1.0,0.0,17.6
1,España,No hubo tentativa,Sí,18-20 años,21-30 años,Andalucía,Pareja en fase de separación,2006,Diciembre,1.0,0.0,17.6
2,España,No hubo tentativa,Sí,21-30 años,16-17 años,Andalucía,Pareja,2023,Febrero,1.0,0.0,17.6
3,España,No hubo tentativa,Sí,21-30 años,21-30 años,Andalucía,Pareja,2012,Octubre,1.0,0.0,17.6
4,España,No hubo tentativa,Sí,31-40 años,21-30 años,Andalucía,Pareja,2010,Septiembre,1.0,0.0,17.6


In [78]:
df_merged.groupby("CCAA").agg({"n_victimas_mortales":"sum","tasa de desempleo %":"median"})

,n_victimas_mortales,tasa de desempleo %
CCAA,,
Andalucía,258.0,17.6
Aragón,33.0,7.9
Asturias,28.0,11.8
Canarias,82.0,16.2
Cantabria,11.0,7.5
Castilla y León,62.0,9.5
Castilla-La Mancha,57.0,12.5
Cataluña,198.0,9.0
Ceuta,3.0,28.4


In [79]:
df_merged.to_csv("Violencia_BI.csv",index=False)